In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

## PART 1: Loading

In [3]:
# Loading the dataset
def load_housing_data(housing_path=''): 
    csv_path = os.path.join(housing_path, "housing.csv") 
    return pd.read_csv(csv_path)
housing = load_housing_data()

# Print dataset function
def print_sample_and_shape(data=housing, sample_size=10):
    print("Data: ",data[:sample_size])
    print("Shape: ",data.shape)

## PART 2: Splitting

In [4]:
DEBUG_P_2 = False

# Adding column based on median_income with less cases
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5) # create category based on median income as an integer using ceil()
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True) # put every sample of income_cat >= 5 into category 5

# Splitting into train and test set
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42) # shuffle and split with representative distribution
for train_index, test_index in split.split(housing, housing["income_cat"]):
    train_set = housing.loc[train_index]
    test_set = housing.loc[test_index]

# Printing dataset
if DEBUG_P_2:
    print_sample_and_shape(train_set)
    print_sample_and_shape(test_set)

# Visualizing the input
if DEBUG_P_2:
    housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)
    plt.show()

# Checking correlation
if DEBUG_P_2:
    corr_matrix = housing.corr()
    print(corr_matrix["median_house_value"].sort_values(ascending=False))

# Print income_cat proportion
# print(housing["income_cat"].value_counts() / len(housing))

# Revert data
for set in (train_set, test_set): 
    set.drop(["income_cat"], axis=1, inplace=True)
housing.drop(["income_cat"], axis=1, inplace=True)

housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

## PART 3: Cleaning

In [5]:
DEBUG_P_3 = False

# (a) Sanitizing

# Replace nulls
from sklearn.impute import SimpleImputer 
imputer = SimpleImputer(strategy="median")
# Extract only number features
housing_num = housing.drop("ocean_proximity", axis=1) # ocean_proximity is a string, only numbers are to be kept
imputer.fit(housing_num)
housing_num_filled = pd.DataFrame(imputer.transform(housing_num), columns=housing_num.columns)

# Encode String as OneHot binary list
# Useful encoders: LabelEncoder (text to num), OneHotEncoder (num to 1Hot), LabelBinarizer (text to 1Hot)
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
housing_cat = encoder.fit_transform(housing["ocean_proximity"])
# print(housing_cat[:10])

# (b) Feature scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
housing_num_scaled = scaler.fit_transform(housing_num_filled)
# print(housing_num_scaled[:10])

# (c) Custom transformers

from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room 
    def fit(self, X, y=None):
        return self # nothing else to do 
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix] 
        population_per_household = X[:, population_ix] / X[:, household_ix] 
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

class DataFrameSelector(BaseEstimator, TransformerMixin): 
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names 
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X[self.attribute_names].values

class FixLabelBinarizer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelBinarizer(*args, **kwargs)
    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self
    def transform(self, x, y=0):
        return self.encoder.transform(x)

# (c) Pipelining
from sklearn.pipeline import Pipeline, FeatureUnion

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

# Numerical pipeline
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)), # Select features
    ('imputer', SimpleImputer(strategy="median")), # Fill nulls
    ('attribs_adder', CombinedAttributesAdder()), # Add combined features
    ('std_scaler', StandardScaler()), # Feature scaling
])

# String pipeline
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)), # Select string features
    ('label_binarizer', FixLabelBinarizer()), # Encode as 1Hot
])

# Union
full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

housing_prepared = full_pipeline.fit_transform(housing)

if DEBUG_P_3:
    print(housing_prepared)

## PART 4: Training

In [6]:
DEBUG_P_4 = False

# Print predictions
def print_predictions_samples(pred, num):
    some_data = housing.iloc[:num]
    some_labels = housing_labels.iloc[:num]
    some_data_prepared = full_pipeline.transform(some_data)
    print("Predictions:\t", pred.predict(some_data_prepared)) 
    print("Labels:\t\t", list(some_labels))

# RMSE
from sklearn.metrics import mean_squared_error
def fit_and_print_rmse(pred):
    pred.fit(housing_prepared, housing_labels)
    housing_predictions = pred.predict(housing_prepared)
    error = np.sqrt(mean_squared_error(housing_labels, housing_predictions))
    print("Error of ",pred," is: ",error)

# Linear regression
from sklearn.linear_model import LinearRegression 
lin_reg = LinearRegression()
if DEBUG_P_4:
    fit_and_print_rmse(lin_reg)

# DecisionTree regression
from sklearn.tree import DecisionTreeRegressor 
tree_reg = DecisionTreeRegressor()
if DEBUG_P_4:
    fit_and_print_rmse(tree_reg)

from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
if DEBUG_P_4:
    fit_and_print_rmse(forest_reg)

# Cross validation score:
# It splits train set into 'cv' folds. 
# Then, for each of them, it performs a fitting taking it out 
# and generates a score taking the current set as the validation set.
from sklearn.model_selection import cross_val_score
def fit_and_print_cv_error(pred):
    scores = cross_val_score(pred, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
    rmse_scores = np.sqrt(-scores)
    print("Scores:", rmse_scores)
    print("Mean:", rmse_scores.mean())
    print("Standard deviation:", rmse_scores.std())

if DEBUG_P_4:
    fit_and_print_cv_error(forest_reg)

## PART 5: Tuning

In [ ]:
DEBUG_P_5 = False

# Hyperparameter tuning
# Useful classes: RandomizedSearchCV, GridSearchCV.
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(housing_prepared, housing_labels)
if DEBUG_P_5:
    print(grid_search.best_params_)
    print(grid_search.best_estimator_)
    cvres = grid_search.cv_results_
    for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]): 
        print(np.sqrt(-mean_score), params)

## PART 6: Testing

In [ ]:
final_model = grid_search.best_estimator_
X_test = test_set.drop("median_house_value", axis=1)
y_test = test_set["median_house_value"].copy()
X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse) # => evaluates to 48,209.6
print("Final RMSE: ", final_rmse)